In [1]:
import numpy as np
import open3d as o3d
import trimesh
import pyrender

from auxiliary import values as v

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
group = 'Gr4'
specimens = v.specimens[group]

source = v.data_path + f'{group}/3DShape/Tissue/2019{specimens[0]}Shape.ply'
target = v.data_path + f'{group}/3DShape/Tissue/2019{specimens[1]}Shape.ply'

source_mesh = trimesh.load(source)
target_mesh = trimesh.load(target)

In [3]:
scene = trimesh.Scene()
scene.add_geometry(source_mesh)
scene.add_geometry(target_mesh)
scene.show()

In [4]:
def trimesh2open3d(tri_mesh):
    mesh = o3d.geometry.TriangleMesh()
    
    mesh.vertices = o3d.utility.Vector3dVector(tri_mesh.vertices)
    mesh.triangles = o3d.utility.Vector3iVector(tri_mesh.faces)
    return mesh

source = trimesh2open3d(source_mesh)
target = trimesh2open3d(target_mesh)

In [5]:
source_pcd = source.sample_points_uniformly(number_of_points=10000)
target_pcd = target.sample_points_uniformly(number_of_points=10000)

In [14]:
source_pcd.estimate_normals()
target_pcd.estimate_normals()

trans_init = np.eye(4)
thr = 1.0

reg_p2p = o3d.pipelines.registration.registration_icp(
    source_pcd, target_pcd, thr, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint()
)

transformation_matrix = reg_p2p.transformation

In [15]:
print(transformation_matrix)

[[ 9.99987178e-01  4.24228425e-03 -2.76547410e-03 -1.02033333e+00]
 [-4.28226584e-03  9.99884018e-01 -1.46154718e-02  4.74790770e+00]
 [ 2.70315037e-03  1.46271269e-02  9.99889364e-01 -6.27814099e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [16]:
source_mesh.apply_transform(transformation_matrix)

<trimesh.Trimesh(vertices.shape=(10002, 3), faces.shape=(20000, 3), name=`20190521_E1Shape.ply`)>

In [18]:
scene = trimesh.Scene()
scene.add_geometry(source_mesh, node_name='source')
scene.add_geometry(target_mesh, node_name='target')
scene.show()